In [ ]:
from dataset import *
output_dataset_folder = "../datasets/pickled"

In [ ]:
from dataset import *
dataset = read_texts('../texts/bajki-extend')
save_tokenized_sentences(dataset, '../datasets/words/bajki-raw.pickle')

In [1]:
from keras.preprocessing.text import Tokenizer
import json
from dataset import *
import re

num_words = 50_000
extra_regex_chars='.,?!:;()-'
token_regex = '„”!"#$%&()*+,-—./:;<=>?@[\\]^_`{|}~0123456789…°'

for c in extra_regex_chars:
    token_regex = token_regex.replace(c, '')

def clear_dataset(dt: List[str], to_left: str = r' A-Za-ząćęłńóśźż\-.,?!:;()\n', to_lower = True) -> List[str]:
    new_dt = [re.sub(f'[^{to_left}]+', '', text.lower() if to_lower else text) for text in dt]
    return new_dt

val = read_texts('../texts/val')
dataset2 = read_texts('../texts/bajki-extend')
dataset1 = read_texts('../texts/books-raw')
dataset = dataset1 + dataset2


# for i in range(len(dataset)):
#     dataset[i] = re.sub(r'(['+extra_regex_chars+'])', r' \1', dataset[i])


# dataset = clear_dataset(dataset)
# val = clear_dataset(val)
# save_tokenized_sentences(dataset, '../datasets/words/books-bajki-raw.pickle')
# save_tokenized_sentences(val, '../datasets/words/books-bajki-raw-val.pickle')

tokenizer = Tokenizer(num_words, token_regex)
tokenizer.fit_on_texts(dataset)
with open(f'../datasets/words/books-bajki-raw-tokenizer_{num_words}.json', 'w', encoding='utf8') as f:
   f.write(json.dumps(tokenizer.to_json(), ensure_ascii=False))


2023-09-09 19:32:54.839035: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-09-09 19:32:55.348608: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [1]:
from keras.layers import TextVectorization
from dataset import *
val = read_texts('../texts/val')

num_words = 100_000
args = {
    "max_tokens": num_words,
    "standardize": "lower_and_strip_punctuation",
    "output_mode": "int",
    "output_sequence_length": 50
}
vec = TextVectorization(**args)

vec.adapt(val)

vec.call(["adrian"])

<tf.Tensor: shape=(1, 50), dtype=int64, numpy=
array([[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0]], dtype=int64)>

In [ ]:
from keras.preprocessing.text import Tokenizer

#Let's add custom sentences 
sentences = [
    "Apples are red",
    "Apples are round",
    "Oranges are round",
    "Grapes are green"
]

#Tokenize the sentences using OOV
myTokenizer = Tokenizer(num_words=100)
myTokenizer.fit_on_texts(sentences)
print(myTokenizer.word_index)

# Unseen Words
test_data = [
    'Grapes are sour but oranges are sweet',
]

test_seq = myTokenizer.texts_to_sequences(test_data)
print("\nTest Sequence = ", test_seq, " => ", [x for x in myTokenizer.sequences_to_texts_generator(test_seq)])

In [ ]:
from tqdm import tqdm
from keras.models import Model, load_model
from lstm import *
from keras.preprocessing.text import tokenizer_from_json
import json 

with open('../datasets/words/books-bajki-raw-tokenizer_100000.json', 'r') as f:
    data = json.load(f)
    tokenizer = tokenizer_from_json(data)
      

model: Model = load_model('../lstm_models/model_best_5.h5')        
text_seed = "dawno temu czerwony kapturek poszedł do lasu"

for temperature in tqdm([0, 0.1, 0.2, 0.3, 0.5, 0.8, 1.0]):
    pr = PredictCallback(text_seed, 50, LEN_MAX_LIMIT, tokenizer, temperature, model)
    text = pr.generate_text()
    with open(f'../generated_texts/lstm/text_temperature_{temperature:.2f}.txt', 'w', encoding='utf8') as f:
        f.write(f'Seed: {text_seed}\n')
        f.write(text)

In [1]:
import numpy as np

a = np.random.randint(0, 15, size=(15, 5))
print(a)
ind = np.asarray([1, 0, 2, 3, 1]) 
ind = np.stack([np.arange(len(ind)), ind]).T
a[ind[:, 0], ind[:, 1]] = 0
a

[[14 11 10  8  5]
 [ 6  2 10  5  0]
 [ 4  4  1 10  3]
 [ 0  2  6  5  4]
 [ 6  0  1  6 14]
 [ 8  3  0 11  2]
 [ 4  1  0  0  0]
 [ 3  0  4  0 13]
 [ 5 10  7 10 12]
 [ 3 14 14  8  2]
 [10  5  2  2  4]
 [12  9  6  1 13]
 [13  2  8  9  8]
 [ 7 13  3  9  7]
 [12 11  3  1 10]]


array([[14,  0, 10,  8,  5],
       [ 0,  2, 10,  5,  0],
       [ 4,  4,  0, 10,  3],
       [ 0,  2,  6,  0,  4],
       [ 6,  0,  1,  6, 14],
       [ 8,  3,  0, 11,  2],
       [ 4,  1,  0,  0,  0],
       [ 3,  0,  4,  0, 13],
       [ 5, 10,  7, 10, 12],
       [ 3, 14, 14,  8,  2],
       [10,  5,  2,  2,  4],
       [12,  9,  6,  1, 13],
       [13,  2,  8,  9,  8],
       [ 7, 13,  3,  9,  7],
       [12, 11,  3,  1, 10]])